In [5]:
import csv
import numpy as np
import pandas as pd

In [6]:
#https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html

In [35]:
unemployment_csv = './data/2019_unemployment.csv'
education_csv = './data/education_2019.csv'
population_csv = './data/nst-est2019-alldata.csv'
snaps_csv = './data/snaps_data_2019.csv'

In [36]:
unemployment_df = pd.read_csv(unemployment_csv)
education_df = pd.read_csv(education_csv)
pop_df = pd.read_csv(population_csv)
pop_df = pop_df[["NAME", "POPESTIMATE2019"]].copy()
snaps_df = pd.read_csv(snaps_csv)

education_df = education_df.rename(columns={"State or Other Area": "State", "2019 Actual": "Education Budget"})
unemployment_df = unemployment_df.rename(columns={"state": "State", "2019 rate": "Unemployment Rate"})
pop_df = pop_df.rename(columns={"NAME": "State", "POPESTIMATE2019": "Population Estimate"})
snaps_df = snaps_df.rename(columns={"state": "State", "household_participants": "SNAPS household_participants",
                                   "individual_participants": "SNAPS individual_participants", "cost": "SNAPS cost",
                                   "cost_per_person": "SNAPS cost_per_person", "cost_per_household": "SNAPS cost_per_household"})

In [47]:
for column in snaps_df:
    if column != "State":
        snaps_df[column] = snaps_df[column].astype(float)
print(snaps_df.head())

ValueError: could not convert string to float: '197,761'

In [38]:
df = pd.merge(education_df, unemployment_df, on='State').drop(columns='rank')
print(df.head())

        State  Education Budget  Unemployment Rate
0     Alabama      2.775338e+09                3.2
1      Alaska      4.005232e+08                5.6
2     Arizona      5.143402e+09                4.8
3    Arkansas      1.350409e+09                3.5
4  California      1.721222e+10                4.1


In [39]:
df = pd.merge(df, pop_df, how='left', on='State')
print(df.head())

        State  Education Budget  Unemployment Rate  Population Estimate
0     Alabama      2.775338e+09                3.2              4903185
1      Alaska      4.005232e+08                5.6               731545
2     Arizona      5.143402e+09                4.8              7278717
3    Arkansas      1.350409e+09                3.5              3017804
4  California      1.721222e+10                4.1             39512223


In [42]:
df = pd.merge(df, snaps_df, how='left', on='State')

In [43]:
df.to_csv('./data/2019-combined.csv', index=False)